In [1]:
import pandas as pd

In [2]:
import geopandas as gpd # type: ignore

def carregar_geojson_em_dataframe(caminho_arquivo):
    try:
        # Carrega o GeoJSON usando o geopandas
        gdf = gpd.read_file(caminho_arquivo)
        return gdf
    except Exception as e:
        print(f"Erro ao abrir o arquivo GeoJSON: {e}")
        return None

acidentes_df = carregar_geojson_em_dataframe("../data/ANTT/BR-101-SC/acidentes_br101sc.geojson")
acidentes_df.head(2)

/home/guibatalhoti/Dados_ANTT/_env/lib/python3.12/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


,n_da_ocorrencia,km,trecho,sentido,tipo_de_acidente,automovel,bicicleta,caminhao,moto,onibus,...,mortos,concessionaria,data_hora,com_vitima,estado,ano,mes,latitude,longitude,geometry
0,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,0,viacosteira,2021-05-01 12:10:00,False,SC,2021,5,-28.425440,-48.857996,POINT (-48.85800 -28.42544)
1,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,0,viacosteira,2021-05-01 12:10:00,False,SC,2021,5,-28.426395,-48.859912,POINT (-48.85991 -28.42639)


In [3]:
# conta a quantidade de valores vazios em latitude
acidentes_df['latitude'].isnull().sum()

539

In [4]:
acidentes_df.columns

Index(['n_da_ocorrencia', 'km', 'trecho', 'sentido', 'tipo_de_acidente',
       'automovel', 'bicicleta', 'caminhao', 'moto', 'onibus', 'outros',
       'tracao_animal', 'transporte_de_cargas_especiais', 'trator_maquinas',
       'utilitarios', 'ilesos', 'levemente_feridos', 'moderadamente_feridos',
       'gravemente_feridos', 'mortos', 'concessionaria', 'data_hora',
       'com_vitima', 'estado', 'ano', 'mes', 'latitude', 'longitude',
       'geometry'],
      dtype='object')

In [5]:
estacoes_proximas = carregar_geojson_em_dataframe("estacoes_inmet.geojson")
estacoes_proximas

/home/guibatalhoti/Dados_ANTT/_env/lib/python3.12/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


,regiao,estado,codigo_estacao,nome_estacao,lat,long,alt,data_fundacao,nome_arquivo,geometry
0,S,RS,MOSTARDAS,A878,-31.248333,-50.906389,3.82,2008-03-11,../data/INMET/2022/INMET_S_RS_A878_MOSTARDAS_0...,POINT (-50.90639 -31.24833)
1,CO,GO,ITUMBIARA,A035,-18.409722,-49.191944,491.17,2007-11-01,../data/INMET/2022/INMET_CO_GO_A035_ITUMBIARA_...,POINT (-49.19194 -18.40972)
2,S,RS,DOM PEDRITO,A881,-31.002500,-54.618056,150.00,2010-04-23,../data/INMET/2022/INMET_S_RS_A881_DOM PEDRITO...,POINT (-54.61806 -31.00250)
3,CO,MT,VILA BELA DA SANTISSIMA TRINDADE,A922,-15.062778,-59.873056,213.00,2006-12-01,../data/INMET/2022/INMET_CO_MT_A922_VILA BELA ...,POINT (-59.87306 -15.06278)
4,SE,MG,ALMENARA,A508,-16.166667,-40.687778,189.11,2002-12-15,../data/INMET/2022/INMET_SE_MG_A508_ALMENARA_0...,POINT (-40.68778 -16.16667)
...,...,...,...,...,...,...,...,...,...,...
2871,S,RS,CAMAQUA,A838,-30.807953,-51.834240,92.30,2006-12-12,../data/INMET/2021/INMET_S_RS_A838_CAMAQUA_01-...,POINT (-51.83424 -30.80795)
2872,NE,BA,BOM JESUS DA LAPA,A418,-13.251111,-43.405278,447.75,2007-05-18,../data/INMET/2021/INMET_NE_BA_A418_BOM JESUS ...,POINT (-43.40528 -13.25111)
2873,N,AM,S. G. DA CACHOEIRA,A134,-0.125207,-67.061246,79.67,2011-08-31,../data/INMET/2021/INMET_N_AM_A134_S. G. DA CA...,POINT (-67.06125 -0.12521)
2874,SE,MG,PASSOS,A516,-20.745237,-46.633916,781.70,2006-07-16,../data/INMET/2021/INMET_SE_MG_A516_PASSOS_01-...,POINT (-46.63392 -20.74524)


In [6]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

def find_nearest_weather_station(accidents_df, weather_stations_df, radius_km=50):
    # Copy the input DataFrames to avoid modifying the original DataFrames
    accidents_df = accidents_df.copy()
    weather_stations_df = weather_stations_df.copy()

    # Remover linhas com valores NaN em coordenadas
    accidents_df = accidents_df.dropna(subset=['latitude', 'longitude'])
    weather_stations_df = weather_stations_df.dropna(subset=['lat', 'long'])

    # Lista para armazenar resultados
    merged_data = []

    # Itera sobre cada linha do DataFrame de acidentes
    for _, accident_row in accidents_df.iterrows():

        accident_location = (accident_row['latitude'], accident_row['longitude'])

        #print(f"Procurando estação climática mais próxima para o acidente em {accident_location}")

        # Calcula a distância para cada estação climática
        weather_stations_df['distance'] = weather_stations_df.apply(
            lambda row: geodesic(accident_location, (row['lat'], row['long'])).kilometers, axis=1
        )

        # Filtra estações dentro do raio especificado
        nearby_stations = weather_stations_df[weather_stations_df['distance'] <= radius_km]

        if not nearby_stations.empty:
            # Encontra a estação mais próxima
            nearest_station = nearby_stations.loc[nearby_stations['distance'].idxmin()]
            combined_row = {**accident_row.to_dict(), **nearest_station.to_dict()}
        else:
            # Se nenhuma estação estiver próxima, preencha com NaN
            combined_row = {**accident_row.to_dict(), **{col: np.nan for col in weather_stations_df.columns}}

        merged_data.append(combined_row)

    # Retorna o DataFrame combinado
    return pd.DataFrame(merged_data)

# acidentes_sample = acidentes_df.sample(15)
# datas_climaticas = pd.read_csv("datas_climaticas.csv")

acidentes_com_estacoes = find_nearest_weather_station(acidentes_df, estacoes_proximas)
acidentes_com_estacoes

KeyboardInterrupt: 

In [ ]:
acidentes_com_estacoes.to_csv("./secundario/acidentes_com_estacoes.csv", index=False)

In [ ]:
import pandas as pd
import os
import re

def load_and_concat_unique_data_with_metadata(df, years=[2020, 2021, 2022, 2023]):
    """
    Lê arquivos CSV baseados em valores únicos na coluna 'nome_arquivo' de um DataFrame,
    processa para anos especificados e concatena sem duplicações, adicionando as informações
    das 8 primeiras linhas como novas colunas.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo a coluna 'nome_arquivo' com caminhos dos arquivos.
        years (list): Lista de anos de interesse para filtrar os dados (default: [2020, 2021, 2022, 2023]).

    Retorna:
        pd.DataFrame: Um DataFrame concatenado sem duplicações, com as colunas extras.
    """
    # Obter valores únicos na coluna 'nome_arquivo'
    unique_files = df['nome_arquivo'].dropna().unique()

    # Inicializa o DataFrame vazio para concatenar os resultados
    concatenated_df = pd.DataFrame()

    # Expressão regular para encontrar o ano no nome do arquivo
    year_pattern = r'\d{4}'  # padrão para encontrar ano de 4 dígitos

    # Conjunto para armazenar arquivos processados e evitar duplicações
    processed_files = set()

    for base_path in unique_files:
        # Encontra o ano presente no nome do arquivo
        match = re.search(year_pattern, base_path)
        if match:
            current_year = match.group()  # Ano encontrado no caminho do arquivo
            
            for year in years:
                # Substitui o ano encontrado no caminho do arquivo pelo ano da iteração
                file_path = base_path.replace(current_year, str(year))

                if file_path not in processed_files and os.path.exists(file_path):
                    print(f"Lendo arquivo: {file_path}")
                    processed_files.add(file_path)  # Marca o arquivo como processado
                    
                    try:
                        # Lê as primeiras 8 linhas do arquivo para extrair as variáveis adicionais
                        with open(file_path, 'r', encoding='latin1') as file:
                            first_8_lines = [next(file) for _ in range(8)]

                        # Extraindo as variáveis a partir das 8 primeiras linhas
                        metadata = {
                            "REGIAO": first_8_lines[0].split(';')[1].strip(),
                            "UF": first_8_lines[1].split(';')[1].strip(),
                            "ESTACAO": first_8_lines[2].split(';')[1].strip(),
                            "CODIGO_WMO": first_8_lines[3].split(';')[1].strip(),
                            "LATITUDE": first_8_lines[4].split(';')[1].strip(),
                            "LONGITUDE": first_8_lines[5].split(';')[1].strip(),
                            "ALTITUDE": first_8_lines[6].split(';')[1].strip(),
                            "DATA_FUNDACAO": first_8_lines[7].split(';')[1].strip()
                        }

                        # Lê o resto do arquivo ignorando as 8 primeiras linhas
                        temp_df = pd.read_csv(
                            file_path,
                            skiprows=8,
                            encoding='latin1',
                            sep=';',  # Especifica o delimitador correto
                            engine='python'         # Usa o motor Python para maior flexibilidade
                        )
                        # Adiciona as variáveis extraídas como novas colunas
                        for key, value in metadata.items():
                            temp_df[key] = value

                        # Adiciona o arquivo lido ao DataFrame concatenado
                        concatenated_df = pd.concat([concatenated_df, temp_df], ignore_index=True)
                    except Exception as e:
                        print(f"Erro ao processar {file_path}: {e}")
                else:
                    print(f"Arquivo já processado ou não encontrado: {file_path}")
        else:
            print(f"Ano não encontrado no arquivo {base_path}")

    # Remove duplicatas do DataFrame final considerando todas as colunas
    concatenated_df = concatenated_df.drop_duplicates()

    return concatenated_df
        
# retira warnings
import warnings
warnings.filterwarnings('ignore')

# Carrega os dados de acidentes com estações climáticas
datas_climaticas = load_and_concat_unique_data_with_metadata(acidentes_com_estacoes)
datas_climaticas.head()

In [ ]:
# # verifica duplicatas
# print(datas_climaticas.duplicated().sum())
# # verifica Data value counts
# print(datas_climaticas['Data'].value_counts())
datas_climaticas

In [ ]:
datas_climaticas.to_csv('./secundario/datas_climaticas.csv', index=False) 

In [ ]:
import pandas as pd
import numpy as np # type: ignore
from geopy.distance import geodesic # type: ignore
from datetime import datetime, timedelta

def find_nearest_weather_station_with_time(accidents_df, weather_stations_df, radius_km=50):
    # Função para converter vírgulas para ponto e depois para float
    def convert_to_float(value):
        if isinstance(value, str):  # Verifica se o valor é uma string
            value = value.replace(',', '.')  # Substitui vírgula por ponto
        return float(value) if value else 0.0  # Converte para float, ou 0.0 se for None ou vazio

    # Copiar os DataFrames de entrada para evitar modificar os originais
    accidents_df = accidents_df.copy()
    weather_stations_df = weather_stations_df.copy()

    # Remover linhas com valores NaN em coordenadas
    accidents_df = accidents_df.dropna(subset=['latitude', 'longitude'])
    weather_stations_df = weather_stations_df.dropna(subset=['LATITUDE', 'LONGITUDE'])

    # Converter as colunas de latitude e longitude para float (com correção de vírgula)
    weather_stations_df['LATITUDE'] = weather_stations_df['LATITUDE'].apply(convert_to_float)
    weather_stations_df['LONGITUDE'] = weather_stations_df['LONGITUDE'].apply(convert_to_float)

    # Converter as colunas de data e hora para datetime
    accidents_df['data_hora'] = pd.to_datetime(accidents_df['data_hora'])
    
    # Ajuste da coluna de hora no dataframe de clima
    weather_stations_df['datetime'] = pd.to_datetime(weather_stations_df['Data'] + ' ' + weather_stations_df['Hora UTC'].str.replace(' UTC', ''), format='%Y/%m/%d %H%M')

    # Lista para armazenar resultados
    merged_data = []

    # Itera sobre cada linha do DataFrame de acidentes
    for _, accident_row in accidents_df.iterrows():
        accident_location = (accident_row['latitude'], accident_row['longitude'])
        accident_time = accident_row['data_hora']

        # print(f"Procurando estação climática mais próxima para o acidente em {accident_location} na hora {accident_time}")

        # Calcula a distância para cada estação climática
        weather_stations_df['distance'] = weather_stations_df.apply(
            lambda row: geodesic(accident_location, (row['LATITUDE'], row['LONGITUDE'])).kilometers, axis=1
        )

        # Filtra estações dentro do raio especificado
        nearby_stations = weather_stations_df[weather_stations_df['distance'] <= radius_km]

        if not nearby_stations.empty:
            # Encontrar a estação mais próxima em termos de data/hora
            nearby_stations['time_diff'] = (nearby_stations['datetime'] - accident_time).abs()

            # Encontra a estação com a menor diferença de tempo
            nearest_station = nearby_stations.loc[nearby_stations['time_diff'].idxmin()]
            
            # Combina os dados do acidente e da estação climática
            combined_row = {**accident_row.to_dict(), **nearest_station.to_dict()}
        else:
            # Se nenhuma estação estiver dentro do raio, preenche com NaN
            combined_row = {**accident_row.to_dict(), **{col: np.nan for col in weather_stations_df.columns}}

        merged_data.append(combined_row)

    # Retorna o DataFrame combinado
    return pd.DataFrame(merged_data)


acidentes_com_estacoes = find_nearest_weather_station_with_time(acidentes_df, datas_climaticas)
acidentes_com_estacoes


In [ ]:
acidentes_com_estacoes.to_csv('./secundario/acidentes_com_estacoes.csv', index=False)

## BATALHAS ALO EXECUTA AS CELULAS A PARTIR DAQUI PFVR

In [ ]:
import pandas as pd

data_path = '../data/DATA_FUSION/data/relevantes/acidentes_com_estacoes.csv'
data = pd.read_csv(data_path)
data

,n_da_ocorrencia,km,trecho,sentido,tipo_de_acidente,automovel,bicicleta,caminhao,moto,onibus,...,geometry,regiao,codigo_estacao,nome_estacao,lat,long,alt,data_fundacao,nome_arquivo,distance
0,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,20.314712
1,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,20.252761
2,12,350.0,BR-101/SC,Decrescente,Saída de pista,0,0,1,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,23.206838
3,12,350.0,BR-101/SC,Decrescente,Saída de pista,0,0,1,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,23.300760
4,13,378.0,BR-101/SC,Decrescente,Capotamento,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2022/INMET_S_SC_A814_URUSSANGA_0...,24.571906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67707,110,316.0,BR-101/SC,Decrescente,Queda de moto,1,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,20.252761
67708,36,382.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.49792 -28.931353),S,ARARANGUA,A867,-28.931353,-49.497920,2.00,2008-09-28,../data/INMET/2022/INMET_S_SC_A867_ARARANGUA_0...,26.176624
67709,36,382.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2022/INMET_S_SC_A814_URUSSANGA_0...,26.142459
67710,34,372.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2022/INMET_S_SC_A814_URUSSANGA_0...,22.092962


In [ ]:
data.columns

Index(['n_da_ocorrencia', 'km', 'trecho', 'sentido', 'tipo_de_acidente',
       'automovel', 'bicicleta', 'caminhao', 'moto', 'onibus', 'outros',
       'tracao_animal', 'transporte_de_cargas_especiais', 'trator_maquinas',
       'utilitarios', 'ilesos', 'levemente_feridos', 'moderadamente_feridos',
       'gravemente_feridos', 'mortos', 'concessionaria', 'data_hora',
       'com_vitima', 'estado', 'ano', 'mes', 'latitude', 'longitude',
       'geometry', 'regiao', 'codigo_estacao', 'nome_estacao', 'lat', 'long',
       'alt', 'data_fundacao', 'nome_arquivo', 'distance'],
      dtype='object')

In [6]:
# Retire todas as linhas onde 'ano' é igual a 2024
data = data[data['ano'] != 2024]
data

,n_da_ocorrencia,km,trecho,sentido,tipo_de_acidente,automovel,bicicleta,caminhao,moto,onibus,...,geometry,regiao,codigo_estacao,nome_estacao,lat,long,alt,data_fundacao,nome_arquivo,distance
0,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,20.314712
1,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,20.252761
2,12,350.0,BR-101/SC,Decrescente,Saída de pista,0,0,1,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,23.206838
3,12,350.0,BR-101/SC,Decrescente,Saída de pista,0,0,1,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,23.300760
4,13,378.0,BR-101/SC,Decrescente,Capotamento,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2022/INMET_S_SC_A814_URUSSANGA_0...,24.571906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67707,110,316.0,BR-101/SC,Decrescente,Queda de moto,1,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,20.252761
67708,36,382.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.49792 -28.931353),S,ARARANGUA,A867,-28.931353,-49.497920,2.00,2008-09-28,../data/INMET/2022/INMET_S_SC_A867_ARARANGUA_0...,26.176624
67709,36,382.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2022/INMET_S_SC_A814_URUSSANGA_0...,26.142459
67710,34,372.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2022/INMET_S_SC_A814_URUSSANGA_0...,22.092962


In [17]:
import re

def ajustar_ano_nome_arquivo(df):
    """
    Ajusta o ano presente na coluna 'nome_arquivo' de acordo com o valor na coluna 'ano'.
    
    Args:
        df (pd.DataFrame): O DataFrame contendo as colunas 'nome_arquivo' e 'ano'.
    
    Returns:
        pd.DataFrame: O DataFrame com a coluna 'nome_arquivo' ajustada.
    """
    def substituir_ano(nome_arquivo, ano_atual):
        if isinstance(nome_arquivo, str):  # Garante que só processamos strings
            return re.sub(r'(\d{4})', lambda x: str(ano_atual), nome_arquivo, count=3)
        return nome_arquivo  # Retorna o valor original caso não seja string
    
    # Aplica a função de substituição para cada linha
    df['nome_arquivo'] = df.apply(lambda row: substituir_ano(row['nome_arquivo'], row['ano']), axis=1)
    
    return df

data = ajustar_ano_nome_arquivo(data)
data

C:\Users\Lucas\AppData\Local\Temp\ipykernel_11244\56621951.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nome_arquivo'] = df.apply(lambda row: substituir_ano(row['nome_arquivo'], row['ano']), axis=1)


,n_da_ocorrencia,km,trecho,sentido,tipo_de_acidente,automovel,bicicleta,caminhao,moto,onibus,...,geometry,regiao,codigo_estacao,nome_estacao,lat,long,alt,data_fundacao,nome_arquivo,distance
0,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2021/INMET_S_SC_A866_Laguna - F...,20.314712
1,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2021/INMET_S_SC_A866_Laguna - F...,20.252761
2,12,350.0,BR-101/SC,Decrescente,Saída de pista,0,0,1,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2021/INMET_S_SC_A866_Laguna - F...,23.206838
3,12,350.0,BR-101/SC,Decrescente,Saída de pista,0,0,1,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2021/INMET_S_SC_A866_Laguna - F...,23.300760
4,13,378.0,BR-101/SC,Decrescente,Capotamento,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2021/INMET_S_SC_A814_URUSSANGA_0...,24.571906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67707,110,316.0,BR-101/SC,Decrescente,Queda de moto,1,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2023/INMET_S_SC_A866_Laguna - F...,20.252761
67708,36,382.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.49792 -28.931353),S,ARARANGUA,A867,-28.931353,-49.497920,2.00,2008-09-28,../data/INMET/2023/INMET_S_SC_A867_ARARANGUA_0...,26.176624
67709,36,382.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2023/INMET_S_SC_A814_URUSSANGA_0...,26.142459
67710,34,372.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2023/INMET_S_SC_A814_URUSSANGA_0...,22.092962


In [18]:
data['nome_arquivo'].unique()

array(['../data/INMET/2021/INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2021_A_31-12-2021.CSV',
       '../data/INMET/2021/INMET_S_SC_A814_URUSSANGA_01-01-2021_A_31-12-2021.CSV',
       '../data/INMET/2021/INMET_S_SC_A806_FLORIANOPOLIS_01-01-2021_A_31-12-2021.CSV',
       nan,
       '../data/INMET/2021/INMET_S_SC_A867_ARARANGUA_01-01-2021_A_31-12-2021.CSV',
       '../data/INMET/2021/INMET_S_RS_A808_TORRES_01-01-2021_A_31-12-2021.CSV',
       '../data/INMET/2022/INMET_S_SC_A814_URUSSANGA_01-01-2022_A_31-12-2022.CSV',
       '../data/INMET/2022/INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2022_A_31-12-2022.CSV',
       '../data/INMET/2022/INMET_S_SC_A867_ARARANGUA_01-01-2022_A_31-12-2022.CSV',
       '../data/INMET/2022/INMET_S_SC_A806_FLORIANOPOLIS_01-01-2022_A_31-12-2022.CSV',
       '../data/INMET/2022/INMET_S_RS_A808_TORRES_01-01-2022_A_31-12-2022.CSV',
       '../data/INMET/2022/INMET_S_SC_A851_ITAPOA_01-01-2022_A_31-12-2022.CSV',
       '../data/INMET/2023/INMET_S

In [12]:
# conta quantos valores vazios tem a coluna data['nome_arquivo']
data['nome_arquivo'].isnull().sum()

645

In [38]:
# Filtre as linhas onde 'nome_arquivo' não é nulo
data = data.dropna(subset=['nome_arquivo'])
data

,n_da_ocorrencia,km,trecho,sentido,tipo_de_acidente,automovel,bicicleta,caminhao,moto,onibus,...,geometry,regiao,codigo_estacao,nome_estacao,lat,long,alt,data_fundacao,nome_arquivo,distance
0,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2021/INMET_S_SC_A866_Laguna - F...,20.314712
1,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2021/INMET_S_SC_A866_Laguna - F...,20.252761
2,12,350.0,BR-101/SC,Decrescente,Saída de pista,0,0,1,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2021/INMET_S_SC_A866_Laguna - F...,23.206838
3,12,350.0,BR-101/SC,Decrescente,Saída de pista,0,0,1,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2021/INMET_S_SC_A866_Laguna - F...,23.300760
4,13,378.0,BR-101/SC,Decrescente,Capotamento,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2021/INMET_S_SC_A814_URUSSANGA_0...,24.571906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67707,110,316.0,BR-101/SC,Decrescente,Queda de moto,1,0,0,0,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2023/INMET_S_SC_A866_Laguna - F...,20.252761
67708,36,382.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.49792 -28.931353),S,ARARANGUA,A867,-28.931353,-49.497920,2.00,2008-09-28,../data/INMET/2023/INMET_S_SC_A867_ARARANGUA_0...,26.176624
67709,36,382.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2023/INMET_S_SC_A814_URUSSANGA_0...,26.142459
67710,34,372.0,BR-101/SC,Decrescente,Saída de pista,1,0,0,0,0,...,POINT (-49.31527777 -28.53249999),S,URUSSANGA,A814,-28.532500,-49.315278,40.56,2008-05-29,../data/INMET/2023/INMET_S_SC_A814_URUSSANGA_0...,22.092962


In [21]:
estacao_climatica = pd.read_csv('../data/INMET/2021/INMET_S_SC_A867_ARARANGUA_01-01-2021_A_31-12-2021.CSV', sep=';', encoding='latin1', skiprows=8)
estacao_climatica

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2021/01/01,0000 UTC,0,1014,1014,"1013,1",NaN,"23,3","18,7","23,5","23,3","19,1","18,4",77.0,74.0,75.0,218.0,"4,7","1,9",NaN
1,2021/01/01,0100 UTC,0,"1014,6","1014,7",1014,NaN,"22,7","18,6","23,3","22,7","18,8","18,3",77.0,74.0,77.0,257.0,"3,4","1,9",NaN
2,2021/01/01,0200 UTC,0,"1014,9","1014,9","1014,6",NaN,"22,4","18,4","23,1","22,4","18,7","18,1",78.0,73.0,78.0,259.0,4,1,NaN
3,2021/01/01,0300 UTC,0,"1015,1","1015,2","1014,9",NaN,"20,8","19,6","22,4","20,8","19,8","17,8",93.0,75.0,93.0,275.0,"4,3","2,4",NaN
4,2021/01/01,0400 UTC,0,"1014,5","1015,2","1014,4",NaN,"20,4",20,"20,9","20,4","20,3","19,6",99.0,93.0,97.0,273.0,"4,7",2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2021/12/31,1900 UTC,0,"1005,9","1006,1","1005,8","2178,6","25,4","20,3",27,"25,4","21,8","20,2",74.0,69.0,73.0,72.0,"11,4","6,8",NaN
8756,2021/12/31,2000 UTC,0,"1006,4","1006,4","1005,9","1096,6","25,2",21,"25,7",25,21,"20,3",78.0,73.0,78.0,83.0,"10,8","4,9",NaN
8757,2021/12/31,2100 UTC,0,1007,1007,"1006,4","363,3","23,8","20,8","25,2","23,8","20,8","20,4",83.0,77.0,83.0,75.0,"9,8","5,1",NaN
8758,2021/12/31,2200 UTC,0,"1007,5","1007,5",1007,"7,1",23,"20,9","23,8",23,"20,9","20,8",88.0,83.0,88.0,70.0,"9,1","3,8",NaN


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def adicionar_dados_climaticos(df):
    dados_adicionados = []
    
    for index, row in df.iterrows():
        try:
            
            # faz um log de cada arquivo que está sendo processado do indice atual / indice total
            print(f"Processando arquivo {index + 1} de {len(df)}")

            # Extrair informações da linha
            nome_arquivo = row['nome_arquivo']
            data_hora_ocorrencia = pd.to_datetime(row['data_hora'])
            
            # Carregar o arquivo climático correspondente
            estacao_climatica = pd.read_csv(
                nome_arquivo,
                sep=';',
                encoding='latin1',
                skiprows=8
            )
            
            # Garantir que 'Data' e 'Hora UTC' estão em formatos compatíveis
            estacao_climatica['Data'] = pd.to_datetime(estacao_climatica['Data'], format='%Y/%m/%d')
            estacao_climatica['Hora UTC'] = estacao_climatica['Hora UTC'].str.replace(' UTC', '').astype(int)
            
            # Criar uma coluna datetime na tabela climática
            estacao_climatica['data_hora_clima'] = estacao_climatica.apply(
                lambda x: x['Data'] + timedelta(hours=x['Hora UTC'] // 100),
                axis=1
            )
            
            # Encontrar a linha com o horário mais próximo
            estacao_climatica['diferenca'] = abs(estacao_climatica['data_hora_clima'] - data_hora_ocorrencia)
            linha_mais_proxima = estacao_climatica.loc[estacao_climatica['diferenca'].idxmin()]
            
            # Adicionar os dados climáticos à linha original, mas sem a coluna 'data_hora_clim'
            dados_adicionados.append(linha_mais_proxima)
        
        except Exception as e:
            print(f"Erro ao processar o arquivo {nome_arquivo}: {e}")
            # Em caso de erro, adiciona NaN para manter o índice correto
            dados_adicionados.append(pd.Series(dtype='float64'))
    
    # Concatenar os dados climáticos ao DataFrame original
    dados_climaticos = pd.DataFrame(dados_adicionados).reset_index(drop=True)
    df_concatenado = pd.concat([df.reset_index(drop=True), dados_climaticos], axis=1)
    
    return df_concatenado

data = adicionar_dados_climaticos(data)

# Exibindo o resultado
data

Processando arquivo 1 de 59213
Processando arquivo 2 de 59213
Processando arquivo 3 de 59213
Processando arquivo 4 de 59213
Processando arquivo 5 de 59213
Processando arquivo 6 de 59213
Processando arquivo 7 de 59213
Processando arquivo 8 de 59213
Processando arquivo 9 de 59213
Processando arquivo 10 de 59213
Processando arquivo 11 de 59213
Processando arquivo 12 de 59213
Processando arquivo 13 de 59213
Processando arquivo 14 de 59213
Processando arquivo 15 de 59213
Processando arquivo 16 de 59213
Processando arquivo 19 de 59213
Processando arquivo 20 de 59213
Processando arquivo 21 de 59213
Processando arquivo 22 de 59213
Processando arquivo 23 de 59213
Processando arquivo 24 de 59213
Processando arquivo 27 de 59213
Processando arquivo 28 de 59213
Processando arquivo 31 de 59213
Processando arquivo 32 de 59213
Processando arquivo 33 de 59213
Processando arquivo 34 de 59213
Processando arquivo 35 de 59213
Processando arquivo 36 de 59213
Processando arquivo 37 de 59213
Processando arqui

KeyboardInterrupt: 

In [36]:
sample[['data_hora', 'data_hora_clima', 'Data', 'Hora UTC', 'diferenca']]

,data_hora,data_hora_clima,Data,Hora UTC,diferenca
0,2023-04-08 06:48:00,2023-04-08 07:00:00,2023-04-08,700.0,0 days 00:12:00
1,2021-05-04 11:35:00,2021-05-04 12:00:00,2021-05-04,1200.0,0 days 00:25:00
2,2021-03-18 04:54:00,2021-03-18 05:00:00,2021-03-18,500.0,0 days 00:06:00
3,2023-03-04 02:41:00,2023-03-04 03:00:00,2023-03-04,300.0,0 days 00:19:00
4,2022-09-18 07:32:00,2022-09-18 08:00:00,2022-09-18,800.0,0 days 00:28:00
5,2022-04-03 04:04:00,2022-04-03 04:00:00,2022-04-03,400.0,0 days 00:04:00
6,2021-11-24 05:25:00,2021-11-24 05:00:00,2021-11-24,500.0,0 days 00:25:00
7,2022-08-01 11:20:00,2022-08-01 11:00:00,2022-08-01,1100.0,0 days 00:20:00
8,2021-12-04 04:21:00,2021-12-04 04:00:00,2021-12-04,400.0,0 days 00:21:00
9,2021-09-27 09:04:00,NaT,NaT,NaN,NaT
